In [15]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip


In [233]:
ffmpeg_extract_subclip("clips/lights.mp4", 6, 7, targetname="clips/checklight1.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [158]:
1200/60

20.0

In [159]:
ffmpeg_extract_subclip("main.mp4", 0, 1200, targetname="test1.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [5]:
from openai import OpenAI
client = OpenAI()

audio_file= open("test1.mp4", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

Boom. All right. We're live. Thank you very much for doing this, man. I really appreciate it. I've been absorbing your information and listening to you talk for quite a while now. So it's great to actually meet you. Thanks for having me. My pleasure. My pleasure. You are one of the rare guys that is, you're a big investor, you're deep in the tech world, but yet you seem to have a very balanced perspective in terms of how to live life, as opposed to not just be entirely focused on success and financial success and tech investing, but rather how to live your life in a happy way. That's an odd balance. Yeah. I think the reason why people like hearing me is because it's like if you go to a circus and you see a bear, right, that's kind of interesting, but not that much. If you see a unicycle, that's interesting, but you see a bear on a unicycle, that's really interesting, right? So when you combine things you're not supposed to combine, people get interested. It's like Bruce Lee, right? Str

In [11]:
len(transcription.text)

21692

In [7]:
audio_file= open("test1.mp4", "rb")
transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

In [13]:
transcript

Transcription(text="Boom. All right. We're live. Thank you very much for doing this, man. I really appreciate it. I've been absorbing your information and listening to you talk for quite a while now. So it's great to actually meet you. Thanks for having me. My pleasure. My pleasure. You are one of the rare guys that is, you're a big investor, you're deep in the tech world, but yet you seem to have a very balanced perspective in terms of how to live life, as opposed to not just be entirely focused on success and financial success and tech investing, but rather how to live your life in a happy way. That's an odd balance. Yeah. I think the reason why people like hearing me is because it's like if you go to a circus and you see a bear, right, that's kind of interesting, but not that much. If you see a unicycle, that's interesting, but you see a bear on a unicycle, that's really interesting. Right? So when you combine things you're not supposed to combine, people get interested. It's like B

In [346]:
print(transcript.words)

In [15]:
import pickle
with open('my_variable.pkl', 'wb') as file:
    pickle.dump(transcript, file)
    
    

In [3]:

import pickle
\
# Load the object from the file
with open('my_variable.pkl', 'rb') as file:
    transcript = pickle.load(file)

In [4]:
new_t = []
for item in transcript.words:
    new_t.append({ 'word': item['word'], 'round' : round(item['start'], 2), 'start' : item['start'], 'end': item['end']})

In [5]:
new_t

[{'word': 'Boom',
  'round': 1.8,
  'start': 1.7999999523162842,
  'end': 2.2799999713897705},
 {'word': 'All',
  'round': 2.28,
  'start': 2.2799999713897705,
  'end': 2.640000104904175},
 {'word': 'right',
  'round': 2.64,
  'start': 2.640000104904175,
  'end': 2.7200000286102295},
 {'word': "We're",
  'round': 2.8,
  'start': 2.799999952316284,
  'end': 2.9800000190734863},
 {'word': 'live',
  'round': 2.98,
  'start': 2.9800000190734863,
  'end': 3.259999990463257},
 {'word': 'Thank',
  'round': 3.48,
  'start': 3.4800000190734863,
  'end': 3.859999895095825},
 {'word': 'you',
  'round': 3.86,
  'start': 3.859999895095825,
  'end': 3.940000057220459},
 {'word': 'very',
  'round': 3.94,
  'start': 3.940000057220459,
  'end': 4.119999885559082},
 {'word': 'much',
  'round': 4.12,
  'start': 4.119999885559082,
  'end': 4.179999828338623},
 {'word': 'for',
  'round': 4.18,
  'start': 4.179999828338623,
  'end': 4.440000057220459},
 {'word': 'doing',
  'round': 4.44,
  'start': 4.440000

In [7]:


# Initialize variables
two_minutes = 90  # 2 minutes in seconds
chunks = []
current_chunk = []
start_time = new_t[0]['start']  # Start time of the first word

# Iterate through the list
for item in new_t:
    word, timestamp = item['word'], item['round']
    
    if timestamp - start_time < two_minutes:
        current_chunk.append((word, timestamp))
    else:
        chunks.append(current_chunk)
        current_chunk = [(word, timestamp)]
        start_time = timestamp

# Add the last chunk if it has words
if current_chunk:
    chunks.append(current_chunk)

# Unpack chunks into individual variables
chunk1, chunk2, chunk3 = chunks[:3]  # Adjust the number of chunks based on your need




# Note: This will print only the first 3 chunks. Adjust the code if you need more or fewer chunks.


In [8]:
len(chunks)

14

In [9]:
import asyncio
from openai import AsyncOpenAI
import time
import ast

client = AsyncOpenAI()

async def extract(prev, curr, nextt):
    
    
    output_format = """{
    "topic" : answer
    "start": start_time_in_seconds,
    "end" : end_time_in_seconds
    }"""
    
    input_prompt = f"""

    
    You are given an audio transcript in this format [('hello', 0), ('my', 1), ('name', 2)]

    the words are in consecutive order. The first index of each tuple contains the actual word being spoken and
    second index inside each tuple contains the timestamp at which the word is being spoken.
    
    Your task is to identify interesting, catchy topic being discussed in the transcript and give me the start and end timestamp
    for that topic and the topic name. Remember the duration of the topic being discussed should be between 60-90 seconds.
    
    The start timestamps should be catchy and should build up the initial words for our main topic that comes afterwards.
    The end timestamp should stop as soon as the topic ends, or when it starts to become verbose.
    
    You'll be given 3 values.
    Previous (list) : The transcript of previous timeframe.
    Current (list): The transcript of current timeframe.
    Next (list) : The transcript of next timeframe.
    
    Mainly, you should focus on "Current" transcript but it may contain truncated informations for which you can use
    the "Previous" and "Next" transcript.
    
    If the topic being discussed is incomplete then please add none to all the keys in the final output dictionary
    
    Your output shouldf be in this format
    
    {output_format}
    


    ### Input:
    
    Previous : {prev}
    Current : {curr}
    Next : {nextt}
    

    ### OUTPUT: 

    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent agent that identifies interesting topic being discussed 
        in the text."""},
        {"role": "user", "content": input_prompt},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125')
    return chat_completion.choices[0].message.content




In [10]:

async def generate_batch(chunks):

    tasks = []
    end = (int(len(chunks)/3)*3)
    for ind in range(0,end, 3):
        print(ind)

        prev = chunks[ind]
        curr = chunks[ind+1]
        nextt = chunks[ind+2]

        tasks.append(extract(prev,curr,nextt))


    results = await asyncio.gather(*tasks)
    return results



In [11]:
result = await generate_batch(chunks)

0
3
6
9


In [12]:
new_result = []
for item in result:
    stripped_item = item.strip('```')
    stripped_item = stripped_item.strip('json')
    new_result.append(ast.literal_eval(stripped_item))

In [13]:
new_result

[{'topic': 'The journey of life and the value of continuous learning',
  'start': 92,
  'end': 181},
 {'topic': 'Reading habits and information consumption',
  'start': 362.92,
  'end': 487.02},
 {'topic': 'Challenges of Fame and Celebrity Culture',
  'start': 632,
  'end': 693},
 {'topic': 'Achieving Happiness and Inner Peace',
  'start': 903.76,
  'end': 1044.24}]

In [16]:
for index,item in enumerate(new_result):
    ffmpeg_extract_subclip("Video.mp4", item['start'], item['end'], targetname=f"{index}.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


## shorten video

In [17]:
for index in range(len(new_result)):
    new_result[index]['transcript'] = []

In [18]:
new_result

[{'topic': 'The journey of life and the value of continuous learning',
  'start': 92,
  'end': 181,
  'transcript': []},
 {'topic': 'Reading habits and information consumption',
  'start': 362.92,
  'end': 487.02,
  'transcript': []},
 {'topic': 'Challenges of Fame and Celebrity Culture',
  'start': 632,
  'end': 693,
  'transcript': []},
 {'topic': 'Achieving Happiness and Inner Peace',
  'start': 903.76,
  'end': 1044.24,
  'transcript': []}]

In [19]:

for index, each_result in enumerate(new_result):
    for ind, item in enumerate(new_t):
        if item['round'] >= each_result['start'] and item['round'] <= each_result['end']:
            new_result[index]['transcript'].append(item)
            


In [197]:
new_result

[{'topic': "Life's Arc and Specialization",
  'start': 92.42,
  'end': 181.56,
  'transcript': [{'word': 'you',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.41999816894531},
   {'word': 'know',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.73999786376953},
   {'word': 'thinker',
    'round': 92.74,
    'start': 92.73999786376953,
    'end': 93.5199966430664},
   {'word': 'et',
    'round': 93.64,
    'start': 93.63999938964844,
    'end': 93.73999786376953},
   {'word': 'cetera',
    'round': 93.74,
    'start': 93.73999786376953,
    'end': 93.83999633789062},
   {'word': 'So',
    'round': 94.58,
    'start': 94.58000183105469,
    'end': 94.91999816894531},
   {'word': 'I',
    'round': 94.92,
    'start': 94.91999816894531,
    'end': 95.23999786376953},
   {'word': 'like',
    'round': 95.24,
    'start': 95.23999786376953,
    'end': 95.54000091552734},
   {'word': 'the',
    'round': 95.54,
    'start': 95.54000091552734,
    'end': 96.05

In [20]:
for item in new_result:
    duration = item['end'] - item['start']
    print(duration)

89
124.09999999999997
61
140.48000000000002


In [21]:

async def shorten(transcript, topic):
    
    
    output_format = """{
    "start": start_time_in_seconds, // edited start time
    "end" : end_time_in_seconds, //edited end time
    }"""
    
    input_prompt = f"""

    
    You are given an audio transcript in this format [('hello', 0), ('my', 1), ('name', 2)].
    The topic for this transcript is {topic}

    the words are in consecutive order. 
    
    Each tuple contains a word and the timestamp at which it was spoken. 
    
    Problem: This transcript is well over 90 seconds but I it to be less than 90 seconds
    Your task is to transform this transcript into a concise and engaging shorter version. 
    
    Follow these steps:

    1. Identify a Catchy Opening: Start transcript with an engaging and interesting line. This could be a captivating statement, a provocative question, or a surprising fact related to the topic.

    2. Content Trimming: Continue the transcript until the point where it starts to become verbose or veers off-topic. Make sure to maintain the core message and most intriguing parts of the discussion.

    
    If the topic being discussed is incomplete then please add none to all the keys in the final output dictionary
    
    Your output should be in this format and should only contain the keys in this format and nothing else
    your 
    
    output_format: 
    
    {output_format}
    


    ### Input Transcript:
    {transcript}
    

    ### OUTPUT: 

    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent script editing agent that makes catchy, interesting transcript."""},
        {"role": "user", "content": input_prompt},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125')
    return chat_completion.choices[0].message.content



In [22]:
new_result

[{'topic': 'The journey of life and the value of continuous learning',
  'start': 92,
  'end': 181,
  'transcript': [{'word': 'you',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.41999816894531},
   {'word': 'know',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.73999786376953},
   {'word': 'thinker',
    'round': 92.74,
    'start': 92.73999786376953,
    'end': 93.5199966430664},
   {'word': 'et',
    'round': 93.64,
    'start': 93.63999938964844,
    'end': 93.73999786376953},
   {'word': 'cetera',
    'round': 93.74,
    'start': 93.73999786376953,
    'end': 93.83999633789062},
   {'word': 'So',
    'round': 94.58,
    'start': 94.58000183105469,
    'end': 94.91999816894531},
   {'word': 'I',
    'round': 94.92,
    'start': 94.91999816894531,
    'end': 95.23999786376953},
   {'word': 'like',
    'round': 95.24,
    'start': 95.23999786376953,
    'end': 95.54000091552734},
   {'word': 'the',
    'round': 95.54,
    'start': 95.54000091552

In [172]:

async def generate_batch_shorten(new_result, new_t):

    tasks = []
    
    for item in new_result:

        if item['end'] - item['start'] < 120:
            continue
            
        topic = item['topic']
        
        
        transcript = item['transcript']

        tasks.append(shorten(transcript,topic))


    results = await asyncio.gather(*tasks)
    return results


In [173]:
final_result = await generate_batch_shorten(new_result, new_t)

In [174]:
len(final_result)

1

In [175]:
final_result

['```json\n{\n    "start": 903.76,\n    "end": 1044.98\n}\n```']

In [122]:
res = []
for item in final_result:
    try:
        stripped_item = item.strip('```')
        stripped_item = stripped_item.strip('json')
        res.append(ast.literal_eval(stripped_item))
    except Exception as e:
        print(e)

invalid syntax (<unknown>, line 3)


In [123]:
res

[{'topic': "Life's Arc - From School to Philosopher",
  'start': 92.42,
  'end': 139.16},
 {'topic': 'Approach to Reading and Information Consumption',
  'start': 362.92,
  'end': 479.48},
 {'topic': 'Achieving Work-Life Balance and Inner Peace',
  'start': 903.76,
  'end': 1020.0}]

In [125]:
new_result

[{'topic': "Life's Arc - From School to Philosopher",
  'start': 92.42,
  'end': 139.16,
  'start_index': 258,
  'end_index': 410},
 {'topic': 'Approach to Reading and Information Consumption',
  'start': 362.92,
  'end': 479.48,
  'start_index': 1264,
  'end_index': 1670},
 {'topic': 'Challenges of Fame and Celebrity',
  'start': 633.38,
  'end': 705.36,
  'start_index': 2163,
  'end_index': 2424},
 {'topic': 'Achieving Work-Life Balance and Inner Peace',
  'start': 903.76,
  'end': 1019.98,
  'start_index': 3094,
  'end_index': 3512}]

In [124]:
for index,item in enumerate(new_result):
    ffmpeg_extract_subclip("Video.mp4", item['start'], item['end'], targetname=f"{index}.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


## remove silence

In [23]:
new_result

[{'topic': 'The journey of life and the value of continuous learning',
  'start': 92,
  'end': 181,
  'transcript': [{'word': 'you',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.41999816894531},
   {'word': 'know',
    'round': 92.42,
    'start': 92.41999816894531,
    'end': 92.73999786376953},
   {'word': 'thinker',
    'round': 92.74,
    'start': 92.73999786376953,
    'end': 93.5199966430664},
   {'word': 'et',
    'round': 93.64,
    'start': 93.63999938964844,
    'end': 93.73999786376953},
   {'word': 'cetera',
    'round': 93.74,
    'start': 93.73999786376953,
    'end': 93.83999633789062},
   {'word': 'So',
    'round': 94.58,
    'start': 94.58000183105469,
    'end': 94.91999816894531},
   {'word': 'I',
    'round': 94.92,
    'start': 94.91999816894531,
    'end': 95.23999786376953},
   {'word': 'like',
    'round': 95.24,
    'start': 95.23999786376953,
    'end': 95.54000091552734},
   {'word': 'the',
    'round': 95.54,
    'start': 95.54000091552

In [225]:
#!/usr/bin/env python
#
# Based on a script by Donald Feury
# https://gitlab.com/dak425/scripts/-/blob/master/trim_silenceV2
# https://youtu.be/ak52RXKfDw8

import math
import sys
import subprocess
import os
import shutil
from moviepy.editor import AudioClip, VideoFileClip, concatenate_videoclips


# Iterate over audio to find the non-silent parts. Outputs a list of
# (speaking_start, speaking_end) intervals.
# Args:
#  window_size: (in seconds) hunt for silence in windows of this size
#  volume_threshold: volume below this threshold is considered to be silence
#  ease_in: (in seconds) add this much silence around speaking intervals
def find_speaking(audio_clip, window_size=0.1, volume_threshold=0.01, ease_in=0.25):
    # First, iterate over audio to find all silent windows.
    num_windows = math.floor(audio_clip.end/window_size)
    window_is_silent = []
    for i in range(num_windows):
        s = audio_clip.subclip(i * window_size, (i + 1) * window_size)
        v = s.max_volume()
        window_is_silent.append(v < volume_threshold)

    # Find speaking intervals.
    speaking_start = 0
    speaking_end = 0
    speaking_intervals = []
    for i in range(1, len(window_is_silent)):
        e1 = window_is_silent[i - 1]
        e2 = window_is_silent[i]
        # silence -> speaking
        if e1 and not e2:
            speaking_start = i * window_size
        # speaking -> silence, now have a speaking interval
        if not e1 and e2:
            speaking_end = i * window_size
            new_speaking_interval = [speaking_start - ease_in, speaking_end + ease_in]
            # With tiny windows, this can sometimes overlap the previous window, so merge.
            need_to_merge = len(speaking_intervals) > 0 and speaking_intervals[-1][1] > new_speaking_interval[0]
            if need_to_merge:
                merged_interval = [speaking_intervals[-1][0], new_speaking_interval[1]]
                speaking_intervals[-1] = merged_interval
            else:
                speaking_intervals.append(new_speaking_interval)

    return speaking_intervals


def run():
    # Parse args
    # Input file path
    file_in = '0.mp4'
    # Output file path
    file_out = 'shorten.mp4'

    vid = VideoFileClip(file_in)
    intervals_to_keep = find_speaking(vid.audio)

    print("Keeping intervals: " + str(intervals_to_keep))
    
    keep_clips = [vid.subclip(max(start, 0), end) for [start, end] in intervals_to_keep]

    edited_video = concatenate_videoclips(keep_clips)
    edited_video.write_videofile(file_out,
        fps=60,
        preset='ultrafast',
        codec='libx264',
        temp_audiofile='temp-audio.m4a',
        remove_temp=True,
        audio_codec="aac",
        threads=6
    )

    vid.close()


In [226]:
run()

Keeping intervals: [[-0.25, 73.25]]
Moviepy - Building video shorten.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video shorten.mp4



Moviepy - Done !
Moviepy - video ready shorten.mp4


## combine it for every 5 seconds



In [24]:
new_result[0]['transcript']

[{'word': 'you',
  'round': 92.42,
  'start': 92.41999816894531,
  'end': 92.41999816894531},
 {'word': 'know',
  'round': 92.42,
  'start': 92.41999816894531,
  'end': 92.73999786376953},
 {'word': 'thinker',
  'round': 92.74,
  'start': 92.73999786376953,
  'end': 93.5199966430664},
 {'word': 'et',
  'round': 93.64,
  'start': 93.63999938964844,
  'end': 93.73999786376953},
 {'word': 'cetera',
  'round': 93.74,
  'start': 93.73999786376953,
  'end': 93.83999633789062},
 {'word': 'So',
  'round': 94.58,
  'start': 94.58000183105469,
  'end': 94.91999816894531},
 {'word': 'I',
  'round': 94.92,
  'start': 94.91999816894531,
  'end': 95.23999786376953},
 {'word': 'like',
  'round': 95.24,
  'start': 95.23999786376953,
  'end': 95.54000091552734},
 {'word': 'the',
  'round': 95.54,
  'start': 95.54000091552734,
  'end': 96.05999755859375},
 {'word': 'model',
  'round': 96.06,
  'start': 96.05999755859375,
  'end': 96.33999633789062},
 {'word': 'of',
  'round': 96.34,
  'start': 96.339996

In [25]:
# Function to combine words in 5-second intervals
def combine_words(transcript, interval=3):
    combined_transcript = []
    transcript.sort(key=lambda x: x['start'])
    
    start_time = transcript[0]['start']
    end_time = start_time + interval
    
    current_chunk = {'start': start_time, 'end': end_time, 'words': []}
    
    for item in transcript:
        if item['start'] >= end_time:
            combined_transcript.append(current_chunk)
            start_time = item['start']
            end_time = start_time + interval
            current_chunk = {'start': start_time, 'end': end_time, 'words': []}
        
        current_chunk['words'].append(item['word'])
    
    combined_transcript.append(current_chunk)  # Append the last chunk
    
    return combined_transcript


In [26]:
interval_words = (combine_words(new_result[0]['transcript']))

In [27]:
input_dict = {num: " ".join(item['words']) for num,item in enumerate(combine_words(new_result[0]['transcript']))}

In [28]:
input_dict

{0: 'you know thinker et cetera So I like',
 1: 'the model of life that the ancients had the',
 2: 'Greeks the Romans right where you would start out and',
 3: "when you're young you're just like going to school Then you're going",
 4: "to war Then you're running a business Then you're",
 5: 'supposed to serve in the Senate or the government Then you become a',
 6: "philosopher There's sort of this arc to life where you try",
 7: 'your hand at everything And as one of my',
 8: 'friends says specialization is for insects right',
 9: "So everyone should just be able to do everything And so I don't believe",
 10: 'in this model anymore of trying to focus your life',
 11: "down on one thing You've got one life Just do everything you're going to do",
 12: "I couldn't agree more And I",
 13: 'think that sometimes people find certain success in',
 14: 'whatever the endeavor is and then they think that that is their niche and',
 15: "they stick with it and they never change And they're almost"

In [112]:

async def identify_images(input_dict):
    
    
    output_format = """{
    "start": start_time_in_seconds, // edited start time
    "end" : end_time_in_seconds, //edited end time
    }"""
    
    input_prompt = """ You are dictionary like this 
    
    {'1': "Greeks the Romans right where you would start out and" , '2' : "when you're young you're just like going to school Then you're going" }
    your task is to find out which of these texts can be 
    formed into an meaningful keyword.
    
    For example:
    If the text is : "Greeks the Romans right where you would start out and"
    
    The text can have meaningful keyword like: "Ancient Romans"
    
    So your task is to formulate a simple two word keyword for the for each of keys the dictionary and output it in JSON format
    
    like this:
    
    
    {'1': "Ancient Rome" , '2' : "School Boy" }
    
    You must try to find keyword for each of the keys but if the texts doesn't contain any meaningful information to be turned into meaningful keyword then in such case 
    please output None for that key.

    """

    input_variable = f"""
    
    ### Input Dictionary:
    
    {input_dict}
    """

    messages = [
        {"role": "system", "content": """ You are an Intelligent agent responsible for constructing meaningful title from a given text"""},
        {"role": "user", "content": input_prompt +  input_variable},
    ]
    
    chat_completion =  await client.chat.completions.create(messages=messages, model='gpt-3.5-turbo-0125', response_format={ "type": "json_object" })
    return chat_completion.choices[0].message.content



In [113]:
ans = await identify_images(input_dict)

In [114]:
import json

ans = json.loads(ans)

In [115]:
print(ans)

{'0': None, '1': 'Ancient Wisdom', '2': None, '3': 'School Life', '4': 'Business Journey', '5': None, '6': 'Life Journey', '7': 'Diverse Skills', '8': None, '9': 'Versatile Skills', '10': 'Life Focus', '11': 'Multi-Faceted Life', '12': None, '13': 'Success Pattern', '14': 'Niche Tunnel Vision', '15': 'Fear of Change', '16': None, '17': 'Mountain Climbing', '18': 'Life Challenge', '19': None, '20': 'Aspiring Peak', '21': 'Climbing Persistence', '22': 'Path Transition', '23': 'Fear of Restart', '24': 'Time Constraints', '25': 'New Beginnings', '26': 'Artistic Inspiration', '27': 'Creative Talent', '28': 'Unique Ability'}


In [116]:
type(ans)

dict

In [117]:
for index, item in enumerate(interval_words):
    print(index,ans.get(str(index), None) )
    interval_words[index]['keywords'] = ans.get(str(index), None)

0 None
1 Ancient Wisdom
2 None
3 School Life
4 Business Journey
5 None
6 Life Journey
7 Diverse Skills
8 None
9 Versatile Skills
10 Life Focus
11 Multi-Faceted Life
12 None
13 Success Pattern
14 Niche Tunnel Vision
15 Fear of Change
16 None
17 Mountain Climbing
18 Life Challenge
19 None
20 Aspiring Peak
21 Climbing Persistence
22 Path Transition
23 Fear of Restart
24 Time Constraints
25 New Beginnings
26 Artistic Inspiration
27 Creative Talent
28 Unique Ability


In [118]:
len(interval_words)

29

In [119]:
import random

index_to_find_images = []


for index in range(0, len(interval_words)-1, int(0.2* len(interval_words))):
    number = random.randint(index + 1, index + 5 - 1)
    print(number)
    
    if interval_words[number]['keywords'] is not None:
        index_to_find_images.append({number: interval_words[number]})

    

2
9
14
16
22
28


In [120]:
index_to_find_images

[{9: {'start': 120.73999786376953,
   'end': 123.73999786376953,
   'words': ['So',
    'everyone',
    'should',
    'just',
    'be',
    'able',
    'to',
    'do',
    'everything',
    'And',
    'so',
    'I',
    "don't",
    'believe'],
   'keywords': 'Versatile Skills'}},
 {14: {'start': 136.5800018310547,
   'end': 139.5800018310547,
   'words': ['whatever',
    'the',
    'endeavor',
    'is',
    'and',
    'then',
    'they',
    'think',
    'that',
    'that',
    'is',
    'their',
    'niche',
    'and'],
   'keywords': 'Niche Tunnel Vision'}},
 {22: {'start': 161.63999938964844,
   'end': 164.63999938964844,
   'words': ['going',
    'to',
    'have',
    'to',
    'go',
    'back',
    'down',
    'to',
    'the',
    'bottom',
    'and',
    'look',
    'for',
    'another',
    'path',
    'Nobody'],
   'keywords': 'Path Transition'}},
 {28: {'start': 180.75999450683594,
   'end': 183.75999450683594,
   'words': ['ability'],
   'keywords': 'Unique Ability'}}]

## get_image

In [208]:
import aiohttp
import asyncio
from PIL import Image
from io import BytesIO

async def fetch_image_id(session, keyword):
    url = f"https://api.pexels.com/v1/search?query={keyword}&per_page=1&orientation=portrait&size=small"
    headers = {
        "Authorization": "DS6m37FqnoQRq8JKFy7J0hnmg1yqQvzImxNHRGjhfLgE8qsQtgDi11U9"
    }

    async with session.get(url, headers=headers) as response:
        if response.status == 200:
            # Parse the response JSON asynchronously
            data = await response.json()
            photo_url = data['photos'][0]['src']['portrait']

            # Make a request to get the photo
            async with session.get(photo_url) as photo_response:
                if photo_response.status == 200:
                    # Read the image content
                    photo_data = await photo_response.read()

                    # Open the image
                    image = Image.open(BytesIO(photo_data))

                    # Save the image
                    image.save(f"images/{keyword}.jpeg")
                    print(f"Image for keyword '{keyword}' saved as '{keyword}.jpeg'")
                else:
                    print(f"Failed to retrieve the photo: {photo_response.status}")
        else:
            print(f"Failed to retrieve data for {keyword}: {response.status}")

async def get_image_ids(keywords):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_image_id(session, keyword) for keyword in keywords]
        await asyncio.gather(*tasks)


In [209]:
keywords = []
for item in index_to_find_images:
    for _,val in item.items():
        keywords.append(val['keywords'])

In [210]:
final_images = await get_image_ids(keywords)

Image for keyword 'Versatile Skills' saved as 'Versatile Skills.jpeg'
Image for keyword 'Unique Ability' saved as 'Unique Ability.jpeg'
Image for keyword 'Niche Tunnel Vision' saved as 'Niche Tunnel Vision.jpeg'
Image for keyword 'Path Transition' saved as 'Path Transition.jpeg'


## make zoom in videos from images

In [144]:
index_to_find_images

[{9: {'start': 120.73999786376953,
   'end': 123.73999786376953,
   'words': ['So',
    'everyone',
    'should',
    'just',
    'be',
    'able',
    'to',
    'do',
    'everything',
    'And',
    'so',
    'I',
    "don't",
    'believe'],
   'keywords': 'Versatile Skills'}},
 {14: {'start': 136.5800018310547,
   'end': 139.5800018310547,
   'words': ['whatever',
    'the',
    'endeavor',
    'is',
    'and',
    'then',
    'they',
    'think',
    'that',
    'that',
    'is',
    'their',
    'niche',
    'and'],
   'keywords': 'Niche Tunnel Vision'}},
 {22: {'start': 161.63999938964844,
   'end': 164.63999938964844,
   'words': ['going',
    'to',
    'have',
    'to',
    'go',
    'back',
    'down',
    'to',
    'the',
    'bottom',
    'and',
    'look',
    'for',
    'another',
    'path',
    'Nobody'],
   'keywords': 'Path Transition'}},
 {28: {'start': 180.75999450683594,
   'end': 183.75999450683594,
   'words': ['ability'],
   'keywords': 'Unique Ability'}}]

## first resizing the image

In [205]:

from PIL import Image

def resize_and_crop(image_path,desired_height = 720 ):
    
    img = Image.open(image_path)

#     desired_height = 720  # Adjust this to your desired height
    width, height = img.size
    ratio = height / desired_height
    new_width = int(width / ratio)

    resized_img = img.resize((new_width, desired_height))

# #     resized_image_path = "images/resized.jpg"
#     resized_img.save(image_path)

    
    # Open the image
    image = resized_img

    # Get the original width and height
    width, height = resized_img.size

    # Define the new desired width
    new_width = (9/16)*height  # Adjust this value as needed

    # Calculate the left and right crop positions to maintain aspect ratio
    left = int((width - new_width) / 2)
    right = left + new_width

    # Create the crop box using (left, top, right, bottom)
    crop_box = (left, 0, right, height)

    # Crop the image
    cropped_image = image.crop(crop_box)

    # Save the cropped image (optional)
    cropped_image.save(image_path)
    # Optionally display the resized image
    # resized_img.show()


In [207]:
resize_and_crop('images/Versatile Skills.jpeg')

## crop width

In [198]:
9/16 * 1080

607.5

In [199]:
from PIL import Image

# Open the image
image = Image.open("images/resized.jpg")

# Get the original width and height
width, height = image.size

# Define the new desired width
new_width = (9/16)*height  # Adjust this value as needed

# Calculate the left and right crop positions to maintain aspect ratio
left = int((width - new_width) / 2)
right = left + new_width

# Create the crop box using (left, top, right, bottom)
crop_box = (left, 0, right, height)

# Crop the image
cropped_image = image.crop(crop_box)

# Save the cropped image (optional)
cropped_image.save("images/cropped_image.jpg")

# Alternatively, display the cropped image
# cropped_image.show()


In [129]:
def Zoom(clip,mode='in',position='center',speed=1):
    fps = clip.fps
    duration = clip.duration
    total_frames = int(duration*fps)
    def main(getframe,t):
        frame = getframe(t)
        h,w = frame.shape[:2]
        i = t*fps
        if mode == 'out':
            i = total_frames-i
        zoom = 1+(i*((0.1*speed)/total_frames))
        positions = {'center':[(w-(w*zoom))/2,(h-(h*zoom))/2],
                     'left':[0,(h-(h*zoom))/2],
                     'right':[(w-(w*zoom)),(h-(h*zoom))/2],
                     'top':[(w-(w*zoom))/2,0],
                     'topleft':[0,0],
                     'topright':[(w-(w*zoom)),0],
                     'bottom':[(w-(w*zoom))/2,(h-(h*zoom))],
                     'bottomleft':[0,(h-(h*zoom))],
                     'bottomright':[(w-(w*zoom)),(h-(h*zoom))]}
        tx,ty = positions[position]
        M = np.array([[zoom,0,tx], [0,zoom,ty]])
        frame = cv2.warpAffine(frame,M,(w,h))
        return frame
    return clip.fl(main)

In [130]:
from moviepy.editor import *
import cv2
import numpy as np

img =  'images/Niche Tunnel Vision.jpeg' #using  the image link above

clip = ImageClip(img).set_fps(30).set_duration(5)
clip = Zoom(clip,mode='in',position='center',speed=1.2) #zoom function above

clip.write_videofile('clips/ggg.mp4',preset='superfast')

Moviepy - Building video clips/ggg.mp4.
Moviepy - Writing video clips/ggg.mp4



Moviepy - Done !
Moviepy - video ready clips/ggg.mp4


In [136]:
key, val = next(iter(index_to_find_images[0].items()))

In [219]:
index_to_find_images[0]

{9: {'start': 120.73999786376953,
  'end': 123.73999786376953,
  'words': ['So',
   'everyone',
   'should',
   'just',
   'be',
   'able',
   'to',
   'do',
   'everything',
   'And',
   'so',
   'I',
   "don't",
   'believe'],
  'keywords': 'Versatile Skills'}}

In [217]:
index_to_find_images[0]

key, val = next(iter(index_to_find_images[0].items()))

In [224]:
from moviepy.editor import AudioFileClip

# Load the audio from another video file
audio_clip = VideoFileClip('subclips/cropped.mp4').subclip(val['start'], val['end']).audio


img = 'images/' + val['keywords'] + '.jpeg'
clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
clip = Zoom(clip,mode='in',position='center',speed=2) #zoom function above
# Set the audio to the image clip
clip = clip.set_audio(audio_clip)
clip.write_videofile(f'subclips/{key}_{key}.mp4',preset='superfast')

Moviepy - Building video subclips/9_9.mp4.
MoviePy - Writing audio in 9_9TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/9_9.mp4



Moviepy - Done !
Moviepy - video ready subclips/9_9.mp4


In [226]:


for item in index_to_find_images:
    key, val = next(iter(item.items()))

    

    # Load the audio from another video file
    audio_clip = VideoFileClip('subclips/cropped.mp4').subclip(val['start'], val['end']).audio


    img = 'images/' + val['keywords'] + '.jpeg'
    
    resize_and_crop(img)
    clip = ImageClip(img).set_fps(30).set_duration(val['end'] - val['start'])
    clip = Zoom(clip,mode='in',position='center',speed=2) #zoom function above
    # Set the audio to the image clip
    clip = clip.set_audio(audio_clip)
    clip.write_videofile(f'subclips/{key}_{key}.mp4',preset='superfast')
    
    

Moviepy - Building video subclips/9_9.mp4.
MoviePy - Writing audio in 9_9TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/9_9.mp4



Moviepy - Done !
Moviepy - video ready subclips/9_9.mp4
Moviepy - Building video subclips/14_14.mp4.
MoviePy - Writing audio in 14_14TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/14_14.mp4



Moviepy - Done !
Moviepy - video ready subclips/14_14.mp4
Moviepy - Building video subclips/22_22.mp4.
MoviePy - Writing audio in 22_22TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/22_22.mp4



Moviepy - Done !
Moviepy - video ready subclips/22_22.mp4
Moviepy - Building video subclips/28_28.mp4.
MoviePy - Writing audio in 28_28TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video subclips/28_28.mp4



Moviepy - Done !
Moviepy - video ready subclips/28_28.mp4


In [149]:
interval_words[10]

{'start': 123.73999786376953,
 'end': 126.73999786376953,
 'words': ['in',
  'this',
  'model',
  'anymore',
  'of',
  'trying',
  'to',
  'focus',
  'your',
  'life'],
 'keywords': 'Life Focus'}

### join these words until the images times.

In [153]:
prev = 0

breaking_points = []
for item in index_to_find_images:
    key, val = next(iter(item.items()))
    
    breaking_points.append((prev,key-1)) #start, end
    breaking_points.append((key,key)) #images
    
    prev = key+1

In [154]:
breaking_points

[(0, 8), (9, 9), (10, 13), (14, 14), (15, 21), (22, 22), (23, 27), (28, 28)]

In [157]:
interval_words[0]

{'start': 92.41999816894531,
 'end': 95.41999816894531,
 'words': ['you', 'know', 'thinker', 'et', 'cetera', 'So', 'I', 'like'],
 'keywords': None}

In [184]:
ffmpeg_extract_subclip("main.mp4", 0, 240, targetname="test1.mp4")

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [186]:
import subprocess

# Define the ffmpeg command
command = ["ffmpeg", "-i", "test1.mp4", "-filter:v", "crop=405:720", "-c:v", "libx264", "-crf", "22", "subclips/cropped.mp4"]

# Execute the command using subprocess.run
subprocess.run(command)

print("FFmpeg command executed successfully!")


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspee

FFmpeg command executed successfully!


[out#0/mp4 @ 0x6000006a4480] video:11603kB audio:3796kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 2.428637%
frame=14388 fps=282 q=-1.0 Lsize=   15773kB time=00:03:59.98 bitrate= 538.4kbits/s dup=1 drop=0 speed= 4.7x    
[libx264 @ 0x14e710810] frame I:64    Avg QP:18.74  size: 18454
[libx264 @ 0x14e710810] frame P:4429  Avg QP:22.40  size:  1849
[libx264 @ 0x14e710810] frame B:9895  Avg QP:27.62  size:   254
[libx264 @ 0x14e710810] consecutive B-frames:  0.6% 12.2% 32.7% 54.5%
[libx264 @ 0x14e710810] mb I  I16..4:  9.0% 63.5% 27.5%
[libx264 @ 0x14e710810] mb P  I16..4:  0.7%  1.6%  0.5%  P16..4: 23.6% 10.1%  3.3%  0.0%  0.0%    skip:60.2%
[libx264 @ 0x14e710810] mb B  I16..4:  0.4%  0.1%  0.1%  B16..8: 16.0%  0.5%  0.0%  direct: 0.1%  skip:82.8%  L0:33.4% L1:66.0% BI: 0.6%
[libx264 @ 0x14e710810] 8x8 transform intra:51.2% inter:83.7%
[libx264 @ 0x14e710810] coded y,uvDC,uvAC intra: 45.4% 32.8% 10.8% inter: 2.5% 1.1% 0.0%
[libx264 @ 0x14e710810] i16 v,h,dc,p: 26

In [188]:
for index, item in enumerate(breaking_points):
    first_start = interval_words[item[0]]['start']
    second_end = interval_words[item[1]]['end']
    
    print(first_start)
    print(second_end)
    
    
    print(index)
    if item[0] != item[1]:
        ffmpeg_extract_subclip("subclips/cropped.mp4", first_start, second_end -2, targetname=f"subclips/{item[0]}_{item[1]}.mp4")
        ffmpeg_extract_subclip("subclips/cropped.mp4", second_end -2, second_end, targetname=f"subclips/end_{item[0]}_{item[1]}.mp4")

92.41999816894531
120.0999984741211
0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
120.73999786376953
123.73999786376953
1
123.73999786376953
136.33999633789062
2
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
136.5800018310547
139.5800018310547
3
139.74000549316406
161.5800018310547
4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
161.63999938964844
164.63999938964844
5
164.72000122070312
180.75999450683594
6
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
180.75999450683594
183.75999450683594
7


## combine last part and images with transition

In [171]:
clip = VideoFileClip('clips/sample1.mp4')

original_width, original_height = clip.size

In [180]:
original_width

1280

In [ ]:
from moviepy.editor import VideoFileClip

def convert_to_portrait(input_path, output_path, target_height):
    # Load the video
    clip = VideoFileClip(input_path)
    
    # Determine the target width for portrait mode
    target_width = clip.size[1]  # The height of the landscape video becomes the width of the portrait video
    
    # Resize the video to have the target width while maintaining aspect ratio
    resized_clip = clip.resize(width=target_width)
    
    # Crop the resized video to the target height
    cropped_clip = resized_clip.crop(
        x_center=resized_clip.w / 2, 
        y_center=resized_clip.h / 2, 
        width=target_width, 
        height=target_height
    )
    
    # Write the output video
    cropped_clip.write_videofile(output_path, codec='libx264')

# Specify input and output paths and target height (e.g., 1280 for 720p to portrait, 1920 for 1080p to portrait)
input_video = "input.mp4"
output_video_720p = "output_720p_portrait.mp4"
output_video_1080p = "output_1080p_portrait.mp4"

# Convert 720p video to 720x1280
convert_to_portrait(input_video, output_video_720p, target_height=1280)

# Convert 1080p video to 1080x1920
convert_to_portrait(input_video, output_video_1080p, target_height=1920)


In [227]:
breaking_points

[(0, 8), (9, 9), (10, 13), (14, 14), (15, 21), (22, 22), (23, 27), (28, 28)]

In [237]:
import moviepy.editor as mpe

def apply_transition(prev,nextt, transition, output):

    clip1 = mpe.VideoFileClip(prev)
    clip2 = mpe.VideoFileClip(nextt)
    transition_clip = mpe.VideoFileClip(transition)

    # Apply mask to remove the green background from the transition clip
    masked_transition = transition_clip.fx(mpe.vfx.mask_color, color=[0, 0, 0], thr=100, s=5)

    # Define the duration of the transition
    transition_duration = masked_transition.duration

    # Ensure the transition does not exceed the duration of the clips
    if transition_duration > clip1.duration or transition_duration > clip2.duration:
        raise ValueError("Transition duration exceeds the duration of one or both video clips.")

    # Calculate the start times for the transition
    transition_start_clip1 = clip1.duration - transition_duration + 0.8
    transition_start_clip2 = transition_duration

    # Set the start time for the masked transition clip
    masked_transition = masked_transition.set_start(transition_start_clip1)

    # Create the composite video with the transition
    composite_transition = mpe.CompositeVideoClip([
        clip1,
        masked_transition,
        clip2.set_start(clip1.duration)
    ]).set_duration(clip1.duration + clip2.duration)


    composite_transition.write_videofile(output, codec='libx264', 
                         audio_codec='aac', preset='superfast')

In [238]:
transition_video = 'clips/lightleaks/checklight1.mp4'

for index,item in enumerate(breaking_points):
    first = item[0]
    second = item[1]
    
    prev = index-1
    
    if first == second:
        prev_video = f'subclips/end_{breaking_points[prev][0]}_{breaking_points[prev][1]}.mp4'
        nextt_video = f'subclips/{first}_{second}.mp4'
        output = f'subclips/{first}_{second}_new.mp4'

        print('Combinig ',prev_video, nextt_video)
        apply_transition(prev_video,nextt_video, transition_video, output)
    
    

Combinig  subclips/end_0_8.mp4 subclips/9_9.mp4
Moviepy - Building video subclips/9_9_new.mp4.
MoviePy - Writing audio in 9_9_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/9_9_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/9_9_new.mp4
Combinig  subclips/end_10_13.mp4 subclips/14_14.mp4
Moviepy - Building video subclips/14_14_new.mp4.
MoviePy - Writing audio in 14_14_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/14_14_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/14_14_new.mp4
Combinig  subclips/end_15_21.mp4 subclips/22_22.mp4
Moviepy - Building video subclips/22_22_new.mp4.
MoviePy - Writing audio in 22_22_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/22_22_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/22_22_new.mp4
Combinig  subclips/end_23_27.mp4 subclips/28_28.mp4
Moviepy - Building video subclips/28_28_new.mp4.
MoviePy - Writing audio in 28_28_newTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/28_28_new.mp4



Moviepy - Done !
Moviepy - video ready subclips/28_28_new.mp4


## combine the video

In [239]:
breaking_points

[(0, 8), (9, 9), (10, 13), (14, 14), (15, 21), (22, 22), (23, 27), (28, 28)]

In [248]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

# List of video file paths to be concatenated
video_files = []

for item in breaking_points:
    first = item[0]
    second = item[1]
    
    
    if first == second:
        video_files.append(f'subclips/{first}_{second}_new.mp4')
    else:
        video_files.append(f'subclips/{first}_{second}.mp4')



# Load each video file into a VideoFileClip object
clips = [VideoFileClip(video) for video in video_files]

# Concatenate the list of video clips into a single video
final_clip = concatenate_videoclips(clips, method="compose")

# Write the final video file
final_clip.write_videofile("subclips/final_video1.mp4", preset='superfast', codec='libx264', audio_codec='aac' )


Moviepy - Building video subclips/final_video1.mp4.
MoviePy - Writing audio in final_video1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video subclips/final_video1.mp4



Moviepy - Done !
Moviepy - video ready subclips/final_video1.mp4


In [243]:
video_files

['subfiles/0_8.mp4',
 'subfiles/9_9_new.mp4',
 'subfiles/10_13.mp4',
 'subfiles/14_14_new.mp4',
 'subfiles/15_21.mp4',
 'subfiles/22_22_new.mp4',
 'subfiles/23_27.mp4',
 'subfiles/28_28_new.mp4']